In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import joblib

In [3]:
# تحميل الـ dataset
df = pd.read_csv('./asthma_disease_data.csv')
df.head()

# إزالة الأعمدة الغير لازمة
df.drop('PatientID', axis=1, inplace=True)
df.drop('DoctorInCharge', axis=1, inplace=True)

# التحقق من البيانات
df.columns
df.shape
df.info()
df.isnull().sum()
df.duplicated().sum()
df.dtypes
df.describe()
df['Age'].value_counts()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     2392 non-null   int64  
 1   Gender                  2392 non-null   int64  
 2   Ethnicity               2392 non-null   int64  
 3   EducationLevel          2392 non-null   int64  
 4   BMI                     2392 non-null   float64
 5   Smoking                 2392 non-null   int64  
 6   PhysicalActivity        2392 non-null   float64
 7   DietQuality             2392 non-null   float64
 8   SleepQuality            2392 non-null   float64
 9   PollutionExposure       2392 non-null   float64
 10  PollenExposure          2392 non-null   float64
 11  DustExposure            2392 non-null   float64
 12  PetAllergy              2392 non-null   int64  
 13  FamilyHistoryAsthma     2392 non-null   int64  
 14  HistoryOfAllergies      2392 non-null   

Age
51    47
12    45
27    43
57    42
22    40
      ..
52    24
68    23
30    23
55    22
9     21
Name: count, Length: 75, dtype: int64

In [4]:
# تقريب القيم زي ما عملت
df['BMI'] = df['BMI'].round(2)
float_columns_to_round = ['PhysicalActivity', 'DietQuality', 'SleepQuality',
                          'PollutionExposure', 'PollenExposure', 'DustExposure']
for col in float_columns_to_round:
    df[col] = df[col].round(0).astype(int)
df['LungFunctionFEV1'] = df['LungFunctionFEV1'].round(1)
df['LungFunctionFVC'] = df['LungFunctionFVC'].round(1)

# التحقق من البيانات بعد التقريب
df[float_columns_to_round]
df[['LungFunctionFEV1', 'LungFunctionFVC']]
df.shape
df.dtypes
df['Diagnosis'].value_counts()





Diagnosis
0    2268
1     124
Name: count, dtype: int64

In [5]:
# Data splitting
X = df.drop(columns=['Diagnosis'])  # Features
y = df['Diagnosis']  # Target variable (Diagnosis)

# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert back to DataFrame
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['Diagnosis'] = y_resampled
df_resampled.shape
df_resampled['Diagnosis'].value_counts()

# تحديد الـ features والـ target بعد SMOTE
X = df_resampled.drop(columns=["Diagnosis"])
y = df_resampled["Diagnosis"]

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
# تدريب الـ scaler على الأعمدة الرقمية بس
numerical_cols = ['BMI', 'LungFunctionFEV1', 'LungFunctionFVC']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# طباعة حجم البيانات
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")

# تعريف دالة تقييم الموديلات


def model_performance_evaluation(models, X_train, y_train, X_test, y_test):
    """
    Evaluates models and prints their performance metrics in a tabular format.

    Parameters:
    - models (dict): A dictionary of model names and the corresponding model object.
    - X_train (DataFrame): The training features.
    - y_train (Series): The target variable for training.
    - X_test (DataFrame): The testing features.
    - y_test (Series): The target variable for testing.

    Returns:
    - None: Prints a table of performance metrics.
    """
    results = []
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        try:
            roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        except AttributeError:
            roc_auc = 'N/A'
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'Confusion Matrix': conf_matrix.tolist(),
            'ROC AUC': roc_auc
        })
    results_df = pd.DataFrame(results)
    print("Model Performance Metrics:\n")
    print(results_df.to_string(index=False))
    return results_df


# Initialize models
models = {
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Neural Network': MLPClassifier(max_iter=500)
}

# Call the evaluation function
model_performance_evaluation(models, X_train, y_train, X_test, y_test)

# Initialize and train Random Forest model with default parameters
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]  # For ROC AUC calculation

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print performance metrics
print("\nRandom Forest Model Performance (Default Parameters):")
print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f"ROC AUC Score: {roc_auc:.6f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# # حفظ الموديل والـ scaler
# joblib.dump(rf, 'ml_service/models/asthma/RandomForest_Asthma-model.pkl')
# joblib.dump(scaler, 'ml_service/models/asthma/scaler.pkl')

# print("Model saved to ml_service/models/asthma/RandomForest_Asthma-model.pkl")
# print("Scaler saved to ml_service/models/asthma/scaler.pkl")

Training set size: (3628, 26), Test set size: (908, 26)


c:\Users\Future Line\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Future Line\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Model Performance Metrics:

                 Model  Accuracy  Precision   Recall  F1 Score        Confusion Matrix  ROC AUC
         Random Forest  0.971366   0.975556 0.966960  0.971239  [[443, 11], [15, 439]] 0.991490
   Logistic Regression  0.874449   0.867965 0.883260  0.875546  [[393, 61], [53, 401]] 0.933668
Support Vector Machine  0.805066   0.771037 0.867841  0.816580 [[337, 117], [60, 394]] 0.883100
   K-Nearest Neighbors  0.795154   0.710692 0.995595  0.829358  [[270, 184], [2, 452]] 0.932916
     Gradient Boosting  0.922907   0.906780 0.942731  0.924406  [[410, 44], [26, 428]] 0.974184
              AdaBoost  0.886564   0.860370 0.922907  0.890542  [[386, 68], [35, 419]] 0.948068
        Neural Network  0.932819   0.905155 0.966960  0.935037  [[408, 46], [15, 439]] 0.968576

Random Forest Model Performance (Default Parameters):
Accuracy: 0.966960
Precision: 0.969027
Recall: 0.964758
F1 Score: 0.966887
ROC AUC Score: 0.991510
Confusion Matrix:
[[440  14]
 [ 16 438]]


In [7]:
print(rf.feature_names_in_)

['Age' 'Gender' 'Ethnicity' 'EducationLevel' 'BMI' 'Smoking'
 'PhysicalActivity' 'DietQuality' 'SleepQuality' 'PollutionExposure'
 'PollenExposure' 'DustExposure' 'PetAllergy' 'FamilyHistoryAsthma'
 'HistoryOfAllergies' 'Eczema' 'HayFever' 'GastroesophagealReflux'
 'LungFunctionFEV1' 'LungFunctionFVC' 'Wheezing' 'ShortnessOfBreath'
 'ChestTightness' 'Coughing' 'NighttimeSymptoms' 'ExerciseInduced']


In [8]:
from collections import Counter
# Step 1: Make predictions on the test set using the Random Forest model
predictions = rf.predict(X_test)

# Step 2: Count the number of "Asthma" (1) and "No Asthma" (0) predictions
prediction_counts = Counter(predictions)

# Step 3: Output the counts
asthma_count = prediction_counts[1]
no_asthma_count = prediction_counts[0]

print(f"Number of 'Asthma' predictions: {asthma_count}")
print(f"Number of 'No Asthma' predictions: {no_asthma_count}")
print(f"Total predictions: {len(predictions)}")

Number of 'Asthma' predictions: 452
Number of 'No Asthma' predictions: 456
Total predictions: 908


RandomForest_Asthma-model_IslV.pkl asthma-scaler_IslV.pkl

In [9]:


# Load the trained model and scaler
rf = joblib.load('RandomForest_Asthma-model_IslV.pkl')
scaler = joblib.load('asthma-scaler_IslV.pkl')

# Step 1: Provide new raw input data (before scaling)
# The input must match ALL feature columns used in training, in the correct order
columns = [
    'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI', 'Smoking', 
    'PhysicalActivity', 'DietQuality', 'SleepQuality', 'PollutionExposure', 
    'PollenExposure', 'DustExposure', 'PetAllergy', 'FamilyHistoryAsthma', 
    'HistoryOfAllergies', 'Eczema', 'HayFever', 'GastroesophagealReflux', 
    'LungFunctionFEV1', 'LungFunctionFVC', 'Wheezing', 'ShortnessOfBreath', 
    'ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced'
]

# Input data from the provided row (excluding Diagnosis)
new_data = np.array([[
    54,                   # Age
    0,                    # Gender
    3,                    # Ethnicity
    2,                    # EducationLevel
    37.07955993135331,    # BMI
    0,                    # Smoking
    4.735168554645618,    # PhysicalActivity
    8.214064158947492,    # DietQuality
    7.4835206038697315,   # SleepQuality
    2.794846705391174,    # PollutionExposure
    3.0551388024739334,   # PollenExposure
    9.484013332832276,    # DustExposure
    0,                    # PetAllergy
    0,                    # FamilyHistoryAsthma
    0,                    # HistoryOfAllergies
    0,                    # Eczema
    1,                    # HayFever
    0,                    # GastroesophagealReflux
    1.6859615519953677,   # LungFunctionFEV1
    3.3468766308464106,   # LungFunctionFVC
    1,                    # Wheezing
    0,                    # ShortnessOfBreath
    1,                    # ChestTightness
    1,                    # Coughing
    0,                    # NighttimeSymptoms
    1                     # ExerciseInduced
]])

# Step 2: Create a DataFrame with the same column names as used in training
new_data_df = pd.DataFrame(new_data, columns=columns)

# Step 3: Apply the same preprocessing as in training
# Round BMI to 2 decimal places and LungFunction columns to 1 decimal place
new_data_df['BMI'] = new_data_df['BMI'].round(2)
new_data_df['LungFunctionFEV1'] = new_data_df['LungFunctionFEV1'].round(1)
new_data_df['LungFunctionFVC'] = new_data_df['LungFunctionFVC'].round(1)

# Round PhysicalActivity, DietQuality, SleepQuality, PollutionExposure, PollenExposure, DustExposure to integers
float_columns_to_round = ['PhysicalActivity', 'DietQuality', 'SleepQuality', 
                          'PollutionExposure', 'PollenExposure', 'DustExposure']
for col in float_columns_to_round:
    new_data_df[col] = new_data_df[col].round(0).astype(int)

# Step 4: Scale only the specified numerical columns (BMI, LungFunctionFEV1, LungFunctionFVC)
numerical_cols = ['BMI', 'LungFunctionFEV1', 'LungFunctionFVC']
new_data_df[numerical_cols] = scaler.transform(new_data_df[numerical_cols])

# Step 5: Predict asthma diagnosis
asthma_predict = rf.predict(new_data_df)

# Step 6: Output the prediction
print(f"Predicted Asthma Diagnosis: {'Asthma' if asthma_predict[0] == 1 else 'No Asthma'}")

Predicted Asthma Diagnosis: No Asthma


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Retrain the model with adjusted hyperparameters
rf_adjusted = RandomForestClassifier(
    random_state=42,
    max_depth=10,           # Limit tree depth
    min_samples_split=10,   # Require more samples per split
    min_samples_leaf=5,     # Require more samples per leaf
    class_weight='balanced' # Balance class weights
)

# Fit the model on the training data (from the original code)
# Assuming X_train, y_train are available from the original split
rf_adjusted.fit(X_train, y_train)

# Evaluate on the test set
y_pred_adjusted = rf_adjusted.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("\n=== Adjusted Random Forest Performance ===")
print(accuracy)
print(classification_report(y_test, y_pred_adjusted))

# # Save the adjusted model
# joblib.dump(rf_adjusted, 'RandomForest_Asthma-model_adjusted.pkl')


=== Adjusted Random Forest Performance ===
0.9669603524229075
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       454
           1       0.92      0.94      0.93       454

    accuracy                           0.93       908
   macro avg       0.93      0.93      0.93       908
weighted avg       0.93      0.93      0.93       908



In [11]:
# Step 1: Apply StandardScaler to numerical columns (BMI, LungFunctionFEV1, LungFunctionFVC)
numerical_cols = ['BMI', 'LungFunctionFEV1', 'LungFunctionFVC']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Step 2: Retrain the model with adjusted hyperparameters
rf_adjusted = RandomForestClassifier(
    random_state=42,
    max_depth=10,           # Limit tree depth
    min_samples_split=10,   # Require more samples per split
    min_samples_leaf=5,     # Require more samples per leaf
    class_weight='balanced' # Balance class weights
)

# Fit the model on the training data
rf_adjusted.fit(X_train, y_train)

# Step 3: Evaluate on the test set
y_pred_adjusted = rf_adjusted.predict(X_test)

# Step 4: Calculate accuracy and print performance
accuracy = accuracy_score(y_test, y_pred_adjusted)
print("\n=== Adjusted Random Forest Performance ===")
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred_adjusted))


=== Adjusted Random Forest Performance ===
Accuracy: 0.9306
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       454
           1       0.92      0.94      0.93       454

    accuracy                           0.93       908
   macro avg       0.93      0.93      0.93       908
weighted avg       0.93      0.93      0.93       908



In [12]:
# # Step 5: Save the adjusted model and scaler
# joblib.dump(rf_adjusted, 'RandomForest_Asthma-model_adjusted.pkl')
# joblib.dump(scaler, 'scaler_adjusted.pkl')
# print("Adjusted model saved to ml_service/models/asthma/RandomForest_Asthma-model_adjusted.pkl")
# print("Adjusted scaler saved to ml_service/models/asthma/scaler_adjusted.pkl")

In [13]:
# Load the adjusted model and scaler (saved from part 1)
rf = joblib.load('RandomForest_Asthma-model_adjusted.pkl')
scaler = joblib.load('scaler_adjusted.pkl')

# Step 1: Provide new raw input data (before scaling)
# The input must match ALL feature columns used in training, in the correct order
columns = [
    'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI', 'Smoking', 
    'PhysicalActivity', 'DietQuality', 'SleepQuality', 'PollutionExposure', 
    'PollenExposure', 'DustExposure', 'PetAllergy', 'FamilyHistoryAsthma', 
    'HistoryOfAllergies', 'Eczema', 'HayFever', 'GastroesophagealReflux', 
    'LungFunctionFEV1', 'LungFunctionFVC', 'Wheezing', 'ShortnessOfBreath', 
    'ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced'
]

# Step 1: Provide new raw input data (before scaling)
# The input must match ALL feature columns used in training, in the correct order
new_data = np.array([[
    35,                   # Age
    1,                    # Gender
    0,                    # Ethnicity
    2,                    # EducationLevel
    27.5,                 # BMI
    1,                    # Smoking
    3.5,                  # PhysicalActivity
    5.0,                  # DietQuality
    6.0,                  # SleepQuality
    8.0,                  # PollutionExposure
    7.5,                  # PollenExposure
    6.5,                  # DustExposure
    1,                    # PetAllergy
    1,                    # FamilyHistoryAsthma
    1,                    # HistoryOfAllergies
    0,                    # Eczema
    1,                    # HayFever
    0,                    # GastroesophagealReflux
    2.1,                  # LungFunctionFEV1
    3.8,                  # LungFunctionFVC
    1,                    # Wheezing
    1,                    # ShortnessOfBreath
    1,                    # ChestTightness
    1,                    # Coughing
    1,                    # NighttimeSymptoms
    1                     # ExerciseInduced
]])

# Step 2: Create a DataFrame with the same column names as used in training
new_data_df = pd.DataFrame(new_data, columns=columns)

# Step 3: Apply the same preprocessing as in training
# Round BMI to 2 decimal places and LungFunction columns to 1 decimal place
new_data_df['BMI'] = new_data_df['BMI'].round(2)
new_data_df['LungFunctionFEV1'] = new_data_df['LungFunctionFEV1'].round(1)
new_data_df['LungFunctionFVC'] = new_data_df['LungFunctionFVC'].round(1)

# Round PhysicalActivity, DietQuality, SleepQuality, PollutionExposure, PollenExposure, DustExposure to integers
float_columns_to_round = ['PhysicalActivity', 'DietQuality', 'SleepQuality', 
                          'PollutionExposure', 'PollenExposure', 'DustExposure']
for col in float_columns_to_round:
    new_data_df[col] = new_data_df[col].round(0).astype(int)

# Step 4: Scale only the specified numerical columns (BMI, LungFunctionFEV1, LungFunctionFVC)
numerical_cols = ['BMI', 'LungFunctionFEV1', 'LungFunctionFVC']
new_data_df[numerical_cols] = scaler.transform(new_data_df[numerical_cols])

# Step 5: Predict asthma diagnosis with the adjusted model
asthma_predict = rf_adjusted.predict(new_data_df)

# Step 6: Output the prediction and probabilities for better insight
probabilities = rf_adjusted.predict_proba(new_data_df)
print(f"Predicted Asthma Diagnosis: {'Asthma' if asthma_predict[0] == 1 else 'No Asthma'}")
print(f"Probability of 'No Asthma' (0): {probabilities[0][0]:.4f}")
print(f"Probability of 'Asthma' (1): {probabilities[0][1]:.4f}")

Predicted Asthma Diagnosis: No Asthma
Probability of 'No Asthma' (0): 0.9323
Probability of 'Asthma' (1): 0.0677


In [14]:
# Define multiple samples
samples = [
    [63, 0, 1, 0, 15.848744398517509, 0, 0.8944483090233335, 5.488695584993768, 8.701002733591553, 7.388480566727442, 2.8555777852179687, 0.9743393830919789, 1, 1, 0, 0, 0, 0, 1.3690511997873338, 4.9412056608744575, 0, 0, 1, 0, 0, 1],  # Sample 0
    [77, 1, 0, 3, 33.15135515728724, 0, 8.369617599681602, 7.755179940672062, 5.851264635104737, 6.431191150460654, 1.6294869905086307, 1.9071972016434435, 0, 0, 0, 0, 0, 0, 1.2925224430501925, 3.7143834873086083, 1, 1, 0, 1, 1, 1],  # Sample 1
    [9, 1, 3, 1, 34.96946962586112, 0, 8.293475319059786, 1.3172863614250807, 6.960847232709893, 1.6742376846785867, 9.624349757689508, 9.94985613405871, 0, 1, 0, 0, 1, 1, 2.289334213132939, 2.9836863482330322, 1, 0, 1, 1, 0, 1],   # Sample 2
    [42, 1, 0, 3, 16.224029068610847, 0, 3.426900354482143, 4.134466546914889, 6.687361221911186, 2.4065001334941485, 0.9461358071670567, 7.974179903908956, 1, 0, 0, 0, 0, 0, 3.7790825828241976, 4.811819957956029, 1, 1, 0, 1, 1, 0]   # Sample 3
]

# Convert samples to numpy array
new_data = np.array(samples)

# Create a DataFrame with the same column names as used in training
new_data_df = pd.DataFrame(new_data, columns=columns)

# Apply the same preprocessing as in training
# Round BMI to 2 decimal places and LungFunction columns to 1 decimal place
new_data_df['BMI'] = new_data_df['BMI'].round(2)
new_data_df['LungFunctionFEV1'] = new_data_df['LungFunctionFEV1'].round(1)
new_data_df['LungFunctionFVC'] = new_data_df['LungFunctionFVC'].round(1)

# Round PhysicalActivity, DietQuality, SleepQuality, PollutionExposure, PollenExposure, DustExposure to integers
float_columns_to_round = ['PhysicalActivity', 'DietQuality', 'SleepQuality',
                          'PollutionExposure', 'PollenExposure', 'DustExposure']
for col in float_columns_to_round:
    new_data_df[col] = new_data_df[col].round(0).astype(int)

# Scale only the specified numerical columns (BMI, LungFunctionFEV1, LungFunctionFVC)
numerical_cols = ['BMI', 'LungFunctionFEV1', 'LungFunctionFVC']
new_data_df[numerical_cols] = scaler.transform(new_data_df[numerical_cols])

# Predict asthma diagnosis for all samples
asthma_predictions = rf.predict(new_data_df)
probabilities = rf.predict_proba(new_data_df)

# Output predictions and probabilities for each sample
for i, (pred, prob) in enumerate(zip(asthma_predictions, probabilities)):
    print(f"\nSample {i}:")
    print(f"Predicted Asthma Diagnosis: {'Asthma' if pred == 1 else 'No Asthma'}")
    print(f"Probability of 'No Asthma' (0): {prob[0]:.4f}")
    print(f"Probability of 'Asthma' (1): {prob[1]:.4f}")


Sample 0:
Predicted Asthma Diagnosis: No Asthma
Probability of 'No Asthma' (0): 0.8095
Probability of 'Asthma' (1): 0.1905

Sample 1:
Predicted Asthma Diagnosis: Asthma
Probability of 'No Asthma' (0): 0.4975
Probability of 'Asthma' (1): 0.5025

Sample 2:
Predicted Asthma Diagnosis: No Asthma
Probability of 'No Asthma' (0): 0.7208
Probability of 'Asthma' (1): 0.2792

Sample 3:
Predicted Asthma Diagnosis: No Asthma
Probability of 'No Asthma' (0): 0.8755
Probability of 'Asthma' (1): 0.1245
